In [40]:
import SimpleITK as sitk
import numpy as np
import nrrd
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler #用于数据归一化处理
from scipy.stats import ttest_ind, levene
from sklearn.linear_model import LassoCV
from sklearn.utils import shuffle

In [24]:
nrrd_file=r'D:\桌面\img.nrrd'
data, options=nrrd.read(nrrd_file)
a= [data]
a=np.array(a)
a.shape

(1, 1068, 800)

In [3]:
png_file=r'D:\img.png'
png_image=sitk.ReadImage(png_file)
# png_array=np.array(png_image)
# a= [png_array]
# a=np.array(a)
# out_png_itk=sitk.GetImageFromArray(np.array(a))
sitk.WriteImage(png_image, png_file.split(".png")[0] + ".nrrd")

In [5]:
img_path=r'D:\桌面\影像组学入门及实践\代码\my_code\测试图像\a-1.jpg'
img_1=Image.open(img_path)
img_1_array=np.array(img_1.convert('L'))
array_list=[]
array_list.append(img_1_array)
out_img_nii = sitk.GetImageFromArray(np.array(array_list))
sitk.WriteImage(out_img_nii,r'D:\out_mask.nii.gz')

In [2]:
a=np.load(r'D:\exe_code\书籍资料\Python\影像组学入门及实践\数据\test2\dataset_voc\SegmentationClass\2014_112061_1.npy')

In [3]:
a

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [12]:
abc = pd.read_excel('./nrrd_file/results_1674981840.xlsx')[abc.columns[1:]]
abc

,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,original_shape_Sphericity,original_shape_SurfaceArea,original_shape_SurfaceVolumeRatio,...,lbp-2D_glszm_SmallAreaLowGrayLevelEmphasis,lbp-2D_glszm_ZoneEntropy,lbp-2D_glszm_ZonePercentage,lbp-2D_glszm_ZoneVariance,lbp-2D_ngtdm_Busyness,lbp-2D_ngtdm_Coarseness,lbp-2D_ngtdm_Complexity,lbp-2D_ngtdm_Contrast,lbp-2D_ngtdm_Strength,label
0,72.620142,66.0,68.0,81.841310,81.841310,3022.916667,57.148642,0.161732,6251.327109,2.067979,...,1.059638e-07,-3.203427e-16,0.000326,0.0,0.0,1000000.0,0.0,0.0,0.0,75
1,66.020979,68.0,60.0,71.021124,71.021124,2949.416667,60.058893,0.163261,6092.006601,2.065495,...,1.114080e-07,-3.203427e-16,0.000334,0.0,0.0,1000000.0,0.0,0.0,0.0,75
2,88.651280,67.0,77.0,95.718337,95.718337,4383.416667,64.997086,0.143877,9002.617503,2.053790,...,5.068073e-08,-3.203427e-16,0.000225,0.0,0.0,1000000.0,0.0,0.0,0.0,75
3,204.718714,206.0,200.0,215.745220,215.745220,31914.166667,199.831360,0.075509,64437.849494,2.019099,...,9.727893e-10,-3.203427e-16,0.000031,0.0,0.0,1000000.0,0.0,0.0,0.0,38
4,97.461979,99.0,91.0,100.623059,100.623059,6875.666667,91.126707,0.124621,14031.510630,2.040749,...,2.074463e-08,-3.203427e-16,0.000144,0.0,0.0,1000000.0,0.0,0.0,0.0,38
5,105.302100,86.0,103.0,105.546198,105.546198,7174.416667,88.090373,0.122894,14637.877678,2.040288,...,1.904598e-08,-3.203427e-16,0.000138,0.0,0.0,1000000.0,0.0,0.0,0.0,38
6,190.784178,184.0,155.0,193.659495,193.659495,22699.166667,153.858936,0.084422,45923.748750,2.023147,...,1.918953e-09,-3.203427e-16,0.000044,0.0,0.0,1000000.0,0.0,0.0,0.0,75
7,88.540562,73.0,76.0,92.590496,92.590496,4456.666667,65.445577,0.143190,9146.292814,2.052272,...,4.909863e-08,-3.203427e-16,0.000222,0.0,0.0,1000000.0,0.0,0.0,0.0,38


In [26]:
data = pd.read_excel('./nrrd_file/results_1674981840.xlsx').iloc[:, 22:]
rows, _ = data.shape
data = shuffle(data)
# data = data.fillna(0)  # data中的空数据填充0
X = data[data.columns[:-1]]
y = data['label']
colNames = X.columns
X = X.astype(np.float64)
X = StandardScaler().fit_transform(X)
X = pd.DataFrame(X)
X.columns = colNames

model_pca = PCA(n_components = 0.99)
model_pca.fit(X)
X_new = model_pca.fit_transform(X)
print(X_new.shape)

(8, 7)


In [45]:
data = pd.read_excel('./nrrd_file/results_1674981840.xlsx').iloc[:, 22:]
data_a=data[data['label']==75].iloc[:, :-1]
data_b=data[data['label']!=75].iloc[:, :-1]

index = []
for colName in data_a.columns:
    if levene(data_a[colName], data_b[colName])[1] > 0.05:
        if ttest_ind(data_a[colName], data_b[colName])[1] < 0.05:
            index.append(colName)
    else:
        if ttest_ind(data_a[colName], data_b[colName],equal_var=False)[1] < 0.05:
            index.append(colName)
print(len(index))
print(index)

C:\Users\99563\AppData\Local\Programs\Python\Python37\lib\site-packages\scipy\stats\morestats.py:2603: RuntimeWarning: invalid value encountered in double_scalars
  W = numer / denom


18
['original_shape2D_Sphericity', 'original_firstorder_Median', 'wavelet-HH_gldm_DependenceNonUniformityNormalized', 'wavelet-LL_firstorder_Median', 'square_firstorder_10Percentile', 'square_firstorder_Median', 'square_firstorder_Minimum', 'square_gldm_LargeDependenceHighGrayLevelEmphasis', 'square_gldm_LargeDependenceLowGrayLevelEmphasis', 'square_gldm_LowGrayLevelEmphasis', 'square_gldm_SmallDependenceLowGrayLevelEmphasis', 'squareroot_firstorder_Median', 'logarithm_firstorder_Median', 'logarithm_gldm_LargeDependenceLowGrayLevelEmphasis', 'logarithm_ngtdm_Busyness', 'exponential_firstorder_10Percentile', 'exponential_firstorder_Median', 'exponential_firstorder_Minimum']


In [51]:
alphas = np.logspace(-4,1,50)
print(alphas)

[1.00000000e-04 1.26485522e-04 1.59985872e-04 2.02358965e-04
 2.55954792e-04 3.23745754e-04 4.09491506e-04 5.17947468e-04
 6.55128557e-04 8.28642773e-04 1.04811313e-03 1.32571137e-03
 1.67683294e-03 2.12095089e-03 2.68269580e-03 3.39322177e-03
 4.29193426e-03 5.42867544e-03 6.86648845e-03 8.68511374e-03
 1.09854114e-02 1.38949549e-02 1.75751062e-02 2.22299648e-02
 2.81176870e-02 3.55648031e-02 4.49843267e-02 5.68986603e-02
 7.19685673e-02 9.10298178e-02 1.15139540e-01 1.45634848e-01
 1.84206997e-01 2.32995181e-01 2.94705170e-01 3.72759372e-01
 4.71486636e-01 5.96362332e-01 7.54312006e-01 9.54095476e-01
 1.20679264e+00 1.52641797e+00 1.93069773e+00 2.44205309e+00
 3.08884360e+00 3.90693994e+00 4.94171336e+00 6.25055193e+00
 7.90604321e+00 1.00000000e+01]


In [52]:
data = pd.read_excel('./nrrd_file/results_1674981840.xlsx').iloc[:, 22:]
data = shuffle(data)
data.index = range(len(data))
X = data[data.columns[:-1]]
y = data['label']
X = X.apply(pd.to_numeric, errors='ignore')  # 将数据类型转化为数值型
colNames = X.columns  # 读取特征的名字
X = X.astype(np.float64)
X = StandardScaler().fit_transform(X)
X = pd.DataFrame(X)
X.columns = colNames

In [55]:
alphas = np.logspace(-4, 1, 50)  # alpha就是损失函数的拉姆达， 从10的-4次方到10的一次方均匀地取50个值
model_lassoCV = LassoCV(alphas=alphas, max_iter=100000).fit(X, y)
conf = pd.Series(model_lassoCV.coef_, colNames)
conf

original_shape_Elongation                -6.807771
original_shape_Flatness                   0.000000
original_shape_LeastAxisLength            0.000000
original_shape_MajorAxisLength           -1.139738
original_shape_Maximum2DDiameterColumn    0.952267
                                            ...   
lbp-2D_ngtdm_Busyness                     0.000000
lbp-2D_ngtdm_Coarseness                   0.000000
lbp-2D_ngtdm_Complexity                   0.000000
lbp-2D_ngtdm_Contrast                     0.000000
lbp-2D_ngtdm_Strength                     0.000000
Length: 1046, dtype: float64